In [28]:
!pip install requests
!pip install selenium==3.5.0

In [29]:
## All years from 1993 to 2023
years = list(range(1993,2024))

In [30]:
## empty list of all urls
All_urls = []

In [31]:
## this page contains all the teams list
All_teams_url = "https://www.basketball-reference.com/teams/"

In [ ]:
## all teams abbreviation
team_list = [
    'OKC', 'SEA', 'MEM', 'VAN', 'GSW', 'PHW', 'SFW', 'SAC', 'KCK', 'KCO',
    'CIN', 'ROC', 'DEN', 'DNR', 'HOU', 'SDR', 'LAC', 'SDC', 'BUF', 'DET',
    'FTW', 'NOH', 'NOK', 'NOP', 'PHI', 'SYR', 'ATL', 'STL', 'MLH', 'TRI',
    'NJN', 'BRK', 'NJA', 'NYA', 'CHA', 'CHO', 'CHH', 'LAL', 'MNL', 'UTA',
    'NOJ', 'WAS', 'WSB', 'CAP', 'BAL', 'CHZ', 'MMS', 'MMT', 'MMP', 'NOB',
    'SAS', 'DLC', 'TEX'
]

## sorting the list 
team_list.sort()

## creating the teams urls list 
for team in team_list:
   All_urls.append(All_teams_url + team + "/")


In [33]:
## all teams urls
All_urls

['https://www.basketball-reference.com/teams/ATL/',
 'https://www.basketball-reference.com/teams/BAL/',
 'https://www.basketball-reference.com/teams/BRK/',
 'https://www.basketball-reference.com/teams/BUF/',
 'https://www.basketball-reference.com/teams/CAP/',
 'https://www.basketball-reference.com/teams/CHA/',
 'https://www.basketball-reference.com/teams/CHH/',
 'https://www.basketball-reference.com/teams/CHO/',
 'https://www.basketball-reference.com/teams/CHZ/',
 'https://www.basketball-reference.com/teams/CIN/',
 'https://www.basketball-reference.com/teams/DEN/',
 'https://www.basketball-reference.com/teams/DET/',
 'https://www.basketball-reference.com/teams/DLC/',
 'https://www.basketball-reference.com/teams/DNR/',
 'https://www.basketball-reference.com/teams/FTW/',
 'https://www.basketball-reference.com/teams/GSW/',
 'https://www.basketball-reference.com/teams/HOU/',
 'https://www.basketball-reference.com/teams/KCK/',
 'https://www.basketball-reference.com/teams/KCO/',
 'https://ww

In [36]:
All_urls = ['https://www.basketball-reference.com/teams/VAN/',
 'https://www.basketball-reference.com/teams/WAS/',
 'https://www.basketball-reference.com/teams/WSB/']

In [37]:
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

## to lunch the web chrome driver
driver = webdriver.Chrome(
    executable_path="./chromedriver.exe"
    )

In [38]:
import pandas as pd
from IPython.display import display
import os

## go through all the teams and extract the necessery stats
for url in All_urls:

    parts = url.split("/")
    team_code = parts[-2]

    os.makedirs("All_teams_stats/{}".format(team_code), exist_ok=True)

    team_stats = pd.DataFrame()
    
    for year in years:
        team_stats_per_year = pd.DataFrame()
        team_url = url + "{}.html".format(year)
        driver.get(team_url)
        driver.execute_script("window.scrollTo(1,10000)")
        time.sleep(0.5)
        
        page = driver.page_source
        
        soup = BeautifulSoup(page, 'html.parser')
        
        ## Team and Opponent Stats 
        init_team_and_opponent = soup.find_all("table", {'id':'team_and_opponent'})
        if init_team_and_opponent is not None and len(init_team_and_opponent) > 0:
            
            team_and_opponent = init_team_and_opponent[0]
            team_and_opponent = pd.read_html(str(team_and_opponent))[0]
            team_and_opponent1 = team_and_opponent.iloc[0:1, :]
            team_and_opponent2 = team_and_opponent.iloc[4:5, :]
            team_and_opponent1 = team_and_opponent1.add_suffix('_team')
            team_and_opponent2 = team_and_opponent2.add_suffix('_opponent')
            team_and_opponent1 = team_and_opponent1.reset_index(drop=True)
            team_and_opponent2 = team_and_opponent2.reset_index(drop=True)
            team_stats_per_year = pd.concat([team_and_opponent1, team_and_opponent2], axis=1)
            ##team_and_opponent1.to_csv("All_teams_stats/{0}/team_and_opponent1_{1}.csv".format(team_code, year))
            ##team_and_opponent2.to_csv("All_teams_stats/{0}/team_and_opponent2_{1}.csv".format(team_code, year))

        ## Team Misc
        init_team_misc = soup.find_all("table", {'id':'team_misc'})
        
        if init_team_misc is not None and len(init_team_misc) > 0:
            
            
            init_team_misc_elements = init_team_misc[0].find_all('tr',class_="over_header")
            
            for element in init_team_misc_elements:
                element.decompose()
            
            team_misc = init_team_misc[0]
            team_misc = (pd.read_html(str(team_misc)))[0]
            team_misc = team_misc.iloc[0:1, :]
            team_misc = team_misc.add_suffix('_misc')
            team_misc = team_misc.reset_index(drop=True)
            team_stats_per_year = pd.concat([team_stats_per_year,team_misc], axis=1)
            ##team_misc.to_csv("All_teams_stats/{0}/team_misc_{1}.csv".format(team_code, year))

        if team_stats_per_year is not None and len(team_stats_per_year) > 0:
            team_stats_per_year = team_stats_per_year.drop(columns=team_stats_per_year.columns[team_stats_per_year.columns.str.contains('Unnamed')])
            team_stats_per_year.insert(0, 'Year', year)
            team_stats = pd.concat([team_stats, team_stats_per_year], axis=0)
        
    ## save the stats in a .csv file
    team_stats.to_csv("All_teams_stats/{0}/team_stats.csv".format(team_code))
    